In [1]:
import numpy as np
from PIL import Image, ImageDraw

> sobel operator를 통해 모서리를 검출한다. 

In [2]:
def convolution_intensity(image_pixel):
    """convolution image pixel

    Args:
        image_pixel (2 dimension array): image pixel matrix.

    Returns:
        convol_h (2 dimension array): convolution array
        convol_v (2 dimension array): convolution array
    """
    sobel_x = np.array([[1,0,-1],
                        [2,0,-2],
                        [1,0,-1]])
    sobel_y = np.array([[1,2,1],
                        [0,0,0],
                        [-1,-2,-1]])
    convol_h = []
    convol_v = []
    
    for i in range(len(image_pixel) - 2):
        convol_v_temp = []
        convol_h_temp = []
        
        for j in range(len(image_pixel[0]) - 2):
            convol_h_temp.append(np.sum(image_pixel[i:i+3:, j:j+3:] * sobel_x))
            convol_v_temp.append(np.sum(image_pixel[i:i+3:, j:j+3:] * sobel_y))
        
        convol_h.append(convol_h_temp)
        convol_v.append(convol_v_temp)
    
    convol_h = np.array(convol_h)
    convol_v = np.array(convol_v)

    return convol_v, convol_h

> image pixel을 단순하게 만들기 위해 gaussian filter를 사용한다.

In [3]:
def gaussian(image_pixel):
    """gaussian

    Args:
        image_pixel (2 dimension array): image pixel matrix

    Returns:
        row (2 dimension array): image pixel matrix with gaussian blur
    """
    gaussian_h = np.array([1,2,1]) / 4 
    gaussian_v = np.array([1,2,1]).T / 4
    row = []

    for i in range(len(image_pixel) - 2):
        col = []
        for j in range(len(image_pixel[0]) - 2):
            col_temp = image_pixel[i:i+3:, j:j+3:] @ gaussian_h @ gaussian_v
            col.append(col_temp)
        row.append(col)
    row = np.array(row)
    
    return row

> gaussian filter를 사용해 이미지를 정리하고 R = det(M) - k(trace(M))^2 식에 따라 R을 구하여 edge인지 corner인지 flat인지를 판별한다. 몇 번의 테스트를 거쳐 threshold를 700000000을 사용하면 적절하게 검출해내는 것을 알아냈다.
> 그림을 그리기 위해 corner_boundary 범위 안에 뭉쳐있는 pixel들을 하나로 정리한다.

In [4]:
def corner_detector(image_pixel):
    """corner detector

    Args:
        image_pixel (2 dimension array): image pixel matrix

    Returns:
        corner_coordinate ([x,y] list): corner with [x,y] coordinate
    """
    corner_coordinate = []
    
    convol_h, convol_v = convolution_intensity(image_pixel)
    gaussian_x = gaussian(convol_h)
    gaussian_y = gaussian(convol_v)
    
    dx = gaussian_x*gaussian_x
    dxy = gaussian_x*gaussian_y
    dy = gaussian_y*gaussian_y
    
    dx2 = gaussian(dx)
    dxy2 = gaussian(dxy)
    dy2 = gaussian(dy)
    
    det = dx2 * dy2 - dxy2**2
    trace = (dx2 + dy2)**2
    
    k = 0.06
    rs = det - k * trace
    threshold = 700000000
    
    for i in range(len(rs)):
        for j in range(len(rs[0])):
            if rs[i][j] > threshold:
                corner_coordinate.append([i, j])

    corner_boundary = 20
                    
    for start in corner_coordinate :
        for end in corner_coordinate[:] :
            if (abs(end[0] - start[0]) < corner_boundary and 
                abs(end[1] - start[1]) < corner_boundary and 
                start != end) :
                corner_coordinate.remove(end)
                
    return corner_coordinate

> corner를 그림에 동그라미로 표시한다. 

In [5]:
def draw_corner(image_pixel, coordinate) :
    """check corner in image

    Args:
        image_pixel (2 dimension array): image pixel matrix
        coordinate ([x,y] list): corner with [x,y] coordinate

    Returns:
        image (2 dimension array): image pixel with corner
    """
    re_image = Image.fromarray(image_pixel)
    draw = ImageDraw.Draw(re_image)
    for coor in coordinate :
        draw.ellipse((coor[1], coor[0], 
                      coor[1] + 10, coor[0] + 10),
                     outline='red')
        
    return re_image

> image file을 color scale과 gray scale로 같이 읽는다. 그냥 읽으면 3차원의 RGB로 나오기 때문에 gray scale로 1차원으로 만들고 계산에 사용한다.

In [6]:
image_file = './data/image.png'

image_gray = Image.open(image_file).convert('L')
image_color = Image.open(image_file)
image_gray_pixel = np.array(image_gray)
image_color_pixel = np.array(image_color)

result_coordinate = corner_detector(image_gray_pixel)

result = draw_corner(image_color_pixel, result_coordinate)
result.show()

print(f'corner : {len(result_coordinate)}')